# Analysis of Inpatient Psychiatric Discharge Rates In Florida and California

In [1]:
import pandas as pd
import glob
import os
%matplotlib inline

## Load Florida data

### Identify hospitals licensed to receive Baker Act patients

In [2]:
FL_AHCA_NUM_COL = "AHCA Number (File Number)"
fl_baker_act_facilities = pd.read_csv("../data/fl/ahca-baker-act-hospitals.csv")\
    .pipe(lambda x: x[x["Facility Type"].notnull()])\
    .set_index(FL_AHCA_NUM_COL)
len(fl_baker_act_facilities)

84

### Identify UHS-owned hospitals

In [3]:
FL_UHS_AHCA_IDS = [
    110068, # Atlantic Shores Hospital 
    23960083, # Central Florida Behavioral Hospital 
    110034, # Emerald Coast Behavioral Hospital 
    104026, # Fort Lauderdale Hospital 
    111520, # Gulf Coast Treatment Center 
    104016, # River Point Behavioral Health 
    110049, # Suncoast Behavioral Health Center 
    110047, # University Behavioral Center 
    23960073, # The Vines Hospital
    23960061, # Wekiva Springs
    104017, # Windmoor Healthcare 
]

In [4]:
fl_baker_act_facilities["is_uhs"] = fl_baker_act_facilities.index.isin(FL_UHS_AHCA_IDS)

### Load discharges

In [5]:
FL_PATH = "~/local-only/florida-inpatient-discharges/data/ahca-data/converted/psych-only.csv"
fl_discharges = pd.read_csv(FL_PATH, low_memory=False, dtype={ "ZIPCODE": str }).set_index("SYS_RECID")

In [6]:
fl_discharges["FACLNBR"].isin(fl_baker_act_facilities.index).mean()

0.98176896357673726

### Combine data, restricting to Baker Act–licensed hospitals

In [7]:
fl_discharges["dschyear"] = fl_discharges["YEAR"].astype(int)

In [8]:
fl_baker_act_discharges = pd.merge(
    fl_discharges[fl_discharges["FACLNBR"].isin(fl_baker_act_facilities.index)],
    fl_baker_act_facilities[[ "Name", "Profit Status", "is_uhs" ]],
    left_on="FACLNBR",
    right_index=True,
    how="left"
)

In [9]:
fl_baker_act_discharges["FACLNBR"].isnull().mean()

0.0

In [10]:
fl_baker_act_discharges["FACLNBR"].nunique()

28

In [11]:
len(fl_baker_act_discharges)

202374

### Classify insurance types

In [12]:
def fl_classify_payer(code):
    if code in [ "L" ]: return "self-pay"
    if code in [ "N" ]: return "non-payment"
    if code in [ "A", "B" ]: return "Medicare"
    if code in [ "C", "D", "H", "I", "J", "K", "O" ]: return "other public"
    if code in [ "E" ]: return "commercial"
    if code in [ "M", "Q" ]: return "other"
    raise ValueError(code)

In [13]:
fl_baker_act_discharges["insurance_type"] = fl_baker_act_discharges["PAYER"].apply(fl_classify_payer)

### Discharges by insurance type

In [14]:
fl_baker_act_discharges[
    fl_baker_act_discharges["Profit Status"].isin([ "For-Profit", "Not-For-Profit" ])
].groupby([
    "Profit Status", "is_uhs", "insurance_type"
]).size().unstack()[[
    "commercial",
    "self-pay",
    "Medicare",
    "other public",
    "non-payment",
    "other"
]]

insurance_type         commercial  self-pay  Medicare  other public  \
Profit Status  is_uhs                                                 
For-Profit     False         9007      3732     19352          9705   
               True         29250      5613     26196         25275   
Not-For-Profit False        14011      9948     18045         26343   

insurance_type         non-payment  other  
Profit Status  is_uhs                      
For-Profit     False           560    697  
               True           2399    517  
Not-For-Profit False          1361    363

In [15]:
fl_baker_act_discharges["state"] = "FL"

## Load California data

### Load hospital information

Limit to hospitals with "Acute Psychiatric" license types.

In [16]:
ca_hospitals = pd.read_excel("../data/ca/Hosp14_util_data_FINAL.xlsx",
    sheetname="Section 1-4",
    skiprows=range(1, 4)
).dropna(subset=["OSHPD_ID"]).pipe(lambda x: x[x["TYPE_LIC"] == "Acute Psychiatric"])
assert(len(ca_hospitals["OSHPD_ID"]) == ca_hospitals["OSHPD_ID"].nunique())

In [17]:
ca_hospitals["Profit Status"] = ca_hospitals["TYPE_CNTRL"].apply({
    "Non-Profit Corporation (incl. Church-related)": "Not-For-Profit",
    "Investor - Corporation": "For-Profit",
    "Investor - Limited Liability Company": "For-Profit",
    "State": "Government",
    "University of California": "Government",
    "City and/or County": "Government"
}.get)

Fix erroneous profit classifications:

In [18]:
# BHC Alhambra and Newport Bay Hospital are miscoded in the directory
ca_hospitals.loc[(ca_hospitals["OSHPD_ID"] == 106190020), "Profit Status"] = "For-Profit"
ca_hospitals.loc[(ca_hospitals["OSHPD_ID"] == 106301304), "Profit Status"] = "For-Profit"

### Identify UHS hospitals

In [19]:
CA_UHS_FACILITY_IDS = [
    106190020, # BHC Alhambra in Rosemead, CA
    106364050, # Canyon Ridge Hospital in Chino, CA
    106190232, # Del Amo Hospital in Torrance, CA
    106014034, # Fremont Hospital in Fremont, CA
    106344021, # Heritage Oaks Hospital in Sacramento, CA
    106342392, # Sierra Vista Hospital in Sacramento, CA
]

In [20]:
ca_hospitals["is_uhs"] = ca_hospitals["OSHPD_ID"].isin(CA_UHS_FACILITY_IDS)

In [21]:
ca_hospitals["oshpd_id"] = ca_hospitals["OSHPD_ID"].astype(int).astype(str).str.slice(3, None)

### Load discharges

In [22]:
CA_NUMERIC_COLS = [ "los", "charge" ]

In [23]:
def load_ca_csv(path):
    with open(path) as f:
        cols = f.readline().strip().split(",")
    coltypes = dict((col, (int if col in CA_NUMERIC_COLS else str))
        for col in cols)
    df = pd.read_csv(path, dtype=coltypes)
    return df[
        df["oshpd_id"].isin(ca_hospitals["oshpd_id"])
    ]

In [24]:
CA_BASE_PATH = os.path.expanduser("~/local-only/ca-inpatient-discharges/data")
CA_CSV_PATHS = glob.glob(CA_BASE_PATH + "/*/DATA FOLDER/Public*/*.txt")

In [25]:
ca_discharges = pd.concat([ load_ca_csv(path) for path in CA_CSV_PATHS ])

In [26]:
ca_discharges["LOSDAYS"] = ca_discharges["los"]

In [27]:
# Each discharge in the data is categorized according to 
# Medicare's “Diagnosis Related Group” (MS-DRG) classification system.
# 
# In California's 2012 data, the field is called `MSDRG`; in 2013 and 2014, it's `msdrg`
ca_discharges["MSDRG"] = ca_discharges["msdrg"].fillna(ca_discharges["MSDRG"]).astype(int)

In [28]:
ca_discharges["dschyear"] = ca_discharges["dschyear"].astype(int)

In [29]:
assert(ca_discharges["oshpd_id"].isin(ca_hospitals["oshpd_id"]).mean() == 1)

### Combine data

In [30]:
ca_discharges = pd.merge(
    ca_discharges,
    ca_hospitals[[ "oshpd_id", "FAC_NAME", "Profit Status", "is_uhs"]],
    on="oshpd_id",
    how="left"
)

### Classify insurance types

In [31]:
ca_discharges["insurance_type"] = ca_discharges["pay_cat"].apply({
        "01": "Medicare",
        "02": "other public",
        "03": "commercial",
        "04": "other public",
        "05": "other public",
        "06": "other public",
        "07": "other public",
        "08": "self-pay",
        "09": "other",
        "00": "other"
}.get)
assert(ca_discharges["insurance_type"].isnull().mean() == 0)

### Discharges by insurance type

In [32]:
ca_discharges[
    ca_discharges["Profit Status"].isin([ "For-Profit", "Not-For-Profit" ])
].groupby([ "Profit Status", "is_uhs", "insurance_type" ])\
    .size().unstack()[[
        "commercial",
        "self-pay",
        "Medicare",
        "other public",
        "other"
    ]]

insurance_type         commercial  self-pay  Medicare  other public  other
Profit Status  is_uhs                                                     
For-Profit     False        39001      2251     23033         20433    899
               True         48479      1057     13602         27499      5
Not-For-Profit False        33664      3488     14440         26107    238

In [33]:
ca_discharges["state"] = "CA"

## Calculate discharge curves

I.e., how many patients were discharged by day X?

In [34]:
def get_discharge_curve(df, max=20):
    return (df["LOSDAYS"].value_counts().sort_index().cumsum() / len(df))\
        .loc[range(max+1)]\
        .ffill().fillna(0)

In [35]:
def get_grouped_dc(df):
    return df[
        df["insurance_type"].isin([ "commercial", "self-pay" ]) &
        df["Profit Status"].isin([ "For-Profit", "Not-For-Profit" ])
    ].groupby([ "Profit Status", "is_uhs", "insurance_type" ])\
        .apply(get_discharge_curve)

### Florida

Below, we examine the demographic composition of commercial-insurance and self-pay patients. Self-paying patients are more likely to  be identified as homeless, and slightly more likely to be a racial or ethnic minority, than commercially-insured patients. The median and mean age for both groups are roughly the same, with self-paying patients being slightly older.

For demographic code definitions, see [Florida's latest data dictionary](https://floridahealthfinderstore.blob.core.windows.net/documents/researchers/OrderData/documents/ED_%20AMB%20Data%20Layout%202015Q4%20March%202016.pdf).

In [36]:
fl_baker_act_discharges["race_is_white"] = fl_baker_act_discharges["RACE"] == 5
fl_baker_act_discharges["ethnicity_is_hispanic"] = fl_baker_act_discharges["ETHNICITY"] == "E1"
fl_baker_act_discharges["is_minority"] = (~fl_baker_act_discharges["race_is_white"]) | fl_baker_act_discharges["ethnicity_is_hispanic"]
fl_baker_act_discharges["is_homeless"] = fl_baker_act_discharges["ZIPCODE"] == "7"

In [37]:
grp = fl_baker_act_discharges[
    fl_baker_act_discharges["insurance_type"].isin([ "commercial", "self-pay", "Medicare" ]) &
    fl_baker_act_discharges["Profit Status"].isin([ "For-Profit", "Not-For-Profit" ])
].groupby("insurance_type")

pd.DataFrame({
    "prop_white": grp["race_is_white"].mean(),
    "prop_hispanic": grp["ethnicity_is_hispanic"].mean(),
    "prop_minority": grp["is_minority"].mean(),
    "prop_homeless": grp["is_homeless"].mean(),
    "median_age": grp["AGE"].median(),
    "mean_age": grp["AGE"].mean().round(1),
}).round(3)

,mean_age,median_age,prop_hispanic,prop_homeless,prop_minority,prop_white
insurance_type,,,,,,
Medicare,51.8,52,0.103,0.024,0.281,0.808
commercial,35.0,33,0.105,0.010,0.256,0.836
self-pay,35.7,34,0.081,0.038,0.274,0.796


Overall discharge rates by hospital profit status and insurance type:

In [38]:
fl_discharge_curves = get_grouped_dc(fl_baker_act_discharges)
fl_discharge_curves.T.head().round(3)

Profit Status  For-Profit                              Not-For-Profit         
is_uhs              False               True                    False         
insurance_type commercial self-pay commercial self-pay     commercial self-pay
LOSDAYS                                                                       
0                   0.008    0.012      0.000    0.000          0.015    0.014
1                   0.060    0.119      0.046    0.151          0.127    0.129
2                   0.164    0.333      0.149    0.338          0.323    0.320
3                   0.300    0.533      0.297    0.551          0.522    0.510
4                   0.439    0.670      0.449    0.691          0.658    0.641

Limited to MS-DRG 885 ("psychoses"), the differences remain:

In [39]:
get_grouped_dc(fl_baker_act_discharges[
    fl_baker_act_discharges["MSDRG"] == 885
]).T.head().round(3)

Profit Status  For-Profit                              Not-For-Profit         
is_uhs              False               True                    False         
insurance_type commercial self-pay commercial self-pay     commercial self-pay
LOSDAYS                                                                       
0                   0.005    0.007      0.000    0.000          0.008    0.010
1                   0.042    0.080      0.035    0.106          0.098    0.095
2                   0.132    0.269      0.127    0.277          0.283    0.253
3                   0.265    0.474      0.269    0.494          0.474    0.435
4                   0.403    0.618      0.411    0.637          0.610    0.563

### California

Looking at the overall discharge rates by hospital profit status and insurance type, we see a similar pattern as in Florida:

In [40]:
ca_discharge_curves = get_grouped_dc(ca_discharges)
ca_discharge_curves.T.head().round(3)

Profit Status  For-Profit                              Not-For-Profit         
is_uhs              False               True                    False         
insurance_type commercial self-pay commercial self-pay     commercial self-pay
LOSDAYS                                                                       
0                   0.004    0.076      0.007    0.021          0.010    0.017
1                   0.045    0.272      0.041    0.141          0.066    0.109
2                   0.156    0.429      0.156    0.336          0.176    0.240
3                   0.344    0.610      0.364    0.528          0.334    0.390
4                   0.492    0.728      0.494    0.654          0.492    0.506

Likewise when limited to MS-DRG 885:

In [41]:
get_grouped_dc(ca_discharges[
    ca_discharges["MSDRG"] == 885
]).T.head().round(3)

Profit Status  For-Profit                              Not-For-Profit         
is_uhs              False               True                    False         
insurance_type commercial self-pay commercial self-pay     commercial self-pay
LOSDAYS                                                                       
0                   0.004    0.091      0.005    0.012          0.008    0.013
1                   0.038    0.306      0.031    0.123          0.058    0.092
2                   0.136    0.456      0.128    0.310          0.174    0.228
3                   0.312    0.633      0.330    0.511          0.336    0.388
4                   0.450    0.734      0.463    0.642          0.474    0.497

---

---

---